In [1]:
import mujoco
import mujoco.viewer
import mediapy as media
import time

In [2]:
# PATHS
HR_MODEL_PATH = "stretch2//stretch.xml"
hr_model = mujoco.MjModel.from_xml_path(HR_MODEL_PATH)


In [3]:

# Load and create renderer
hr_data = mujoco.MjData(hr_model)
hr_renderer = mujoco.Renderer(hr_model)

# Dynamics
mujoco.mj_forward(hr_model, hr_data)
hr_renderer.update_scene(hr_data)
#media.show_image(hr_renderer.render()) # Show image

# Simulation
duration = 3
fps = 60
frames = []
camera_frames = []

# HR controls
mujoco.mj_resetData(hr_model, hr_data)
hr_data.ctrl[0] = 5
hr_data.ctrl[7] = -2
hr_data.ctrl[6] = 1

# Run simulation
scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = False
for i in range(duration*fps):
	while hr_data.time < i/fps:
		# Render simulation
		mujoco.mj_step(hr_model, hr_data)
	hr_renderer.update_scene(hr_data, camera='camera_depth')
	cam_pixels = hr_renderer.render()
	if i == 5:
		print(len(cam_pixels[0]))
		print(cam_pixels)
	camera_frames.append(cam_pixels)

# Show simulation
media.show_video(camera_frames, fps=fps)


320
[[[20 30 41]
  [21 31 42]
  [21 31 42]
  ...
  [21 31 41]
  [21 31 41]
  [21 31 41]]

 [[21 31 42]
  [21 31 42]
  [21 31 42]
  ...
  [21 31 41]
  [21 31 41]
  [21 31 41]]

 [[21 31 42]
  [21 31 42]
  [21 31 42]
  ...
  [21 31 41]
  [21 31 41]
  [21 31 41]]

 ...

 [[10 21 31]
  [10 21 31]
  [10 21 31]
  ...
  [10 21 31]
  [10 21 31]
  [10 21 31]]

 [[10 21 31]
  [10 21 31]
  [10 21 31]
  ...
  [10 21 31]
  [10 21 31]
  [10 21 31]]

 [[10 21 31]
  [10 21 31]
  [10 21 31]
  ...
  [10 21 31]
  [10 21 31]
  [10 21 31]]]


This browser does not support the video tag.


In [10]:
# Stretch 3 Stuff that Ken is working on
HR_URDF_PATH = 'SE3/stretch_description_SE3_eoa_wrist_dw3_tool_sg3.xml'
hr_model = mujoco.MjModel.from_xml_path(HR_URDF_PATH)
hr_data = mujoco.MjData(hr_model)

In [4]:
m = mujoco.MjModel.from_xml_path(HR_MODEL_PATH)
d = mujoco.MjData(m)

r = mujoco.Renderer(m)

scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = False
fps = 60
camera_frames = []

with mujoco.viewer.launch_passive(m, d) as viewer:
  # Close the viewer automatically after 30 wall-seconds.
  start = time.time()
  mujoco.mj_resetData(m, d)

  d.ctrl[0] = 5
  d.ctrl[1]= 5
  d.ctrl[7] = -2
  d.ctrl[6] = 1

  i = 0

  while viewer.is_running() and time.time() - start < 5:
    step_start = time.time()

    # mj_step can be replaced with code that also evaluates
    # a policy and applies a control signal before stepping the physics.
    mujoco.mj_step(m, d)

    # Example modification of a viewer option: toggle contact points every two seconds.
    with viewer.lock():
      viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(d.time % 2)

    # Pick up changes to the physics state, apply perturbations, update options from GUI.
    viewer.sync()

    
    if time.time() - start > i / fps:
      i += 1
      r.update_scene(d, camera='camera_rgb')
      cam_pixels = r.render()
      camera_frames.append(cam_pixels)
    
    # Rudimentary time keeping, will drift relative to wall clock.
    time_until_next_step = m.opt.timestep - (time.time() - step_start)
    if time_until_next_step > 0:
      time.sleep(time_until_next_step)

media.show_video(camera_frames, fps=fps)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


This browser does not support the video tag.
